In [56]:
from transformers import AutoTokenizer, AutoModel, AutoModelForSeq2SeqLM
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm
from nltk.translate.bleu_score import corpus_bleu
import random
from copy import deepcopy
from tqdm.auto import trange

In [8]:
train_path = "en_kr_data/train/ko2en_training_csv/ko2en_finance_1_training.csv"
val_path = "en_kr_data/val/ko2en_validation_csv/ko2en_finance_2_validation.csv" 
train_df = pd.read_csv(train_path, sep=',')
val_test_df = pd.read_csv(val_path, sep=',')
val_df, test_df = val_test_df[:len(val_test_df)//2], val_test_df[len(val_test_df)//2:]
print(len(train_df), len(val_df), len(test_df))

#sort sentences by length
#prune sentences over 20 words
#train set small
#get scores for model

160000 10000 10000


In [11]:
#kor_Hang

name = "facebook/nllb-200-distilled-600M"
tokenizer = AutoTokenizer.from_pretrained(name)
nllb = AutoModelForSeq2SeqLM.from_pretrained(name)

config.json: 100%|██████████| 846/846 [00:00<00:00, 5.63MB/s]
pytorch_model.bin: 100%|██████████| 2.46G/2.46G [01:44<00:00, 23.5MB/s]
generation_config.json: 100%|██████████| 189/189 [00:00<00:00, 771kB/s]
/Users/christine/miniconda3/envs/mt-class/lib/python3.11/site-packages/transformers/generation/utils.py:1369: UserWarning: Using `max_length`'s default (200) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


However, there is a difference of opinion as to whether this trend will continue in the second half.


In [12]:
tokenizer.src_lang = "kor_Hang"
inputs = tokenizer(text="다만 하반기에도 이같은 기조가 이어질 지에 대해서는 의견이 분분하다.", return_tensors="pt")
translated_tokens = nllb.generate(
    **inputs, forced_bos_token_id=tokenizer.lang_code_to_id["eng_Latn"]
)
print(tokenizer.decode(translated_tokens[0], skip_special_tokens=True))

However, there is a difference of opinion as to whether this trend will continue in the second half.


In [20]:
batch_size = 16
test_df_small = test_df[:1000]
#batches = [test_df.iloc[i:i + batch_size] for i in range(0, len(test_df), batch_size)]
batches = [test_df_small.iloc[i:i + batch_size] for i in range(0, len(test_df_small), batch_size)]

english_translations = []

for df_batch in tqdm(batches):
    inputs = tokenizer(text=df_batch['한국어'].tolist(), return_tensors="pt", padding=True, truncation=True)
    translated_tokens = nllb.generate(
        **inputs, forced_bos_token_id=tokenizer.lang_code_to_id["eng_Latn"]
    )
    translated_sentences = tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)
    
    print(translated_sentences)
    english_translations += translated_sentences

df_result = pd.concat(batches, ignore_index=True)
print(df_result)

  2%|▏         | 1/63 [00:07<07:50,  7.59s/it]

["And it's also been analyzed that by the end of the year, time-lockers are working on a kind of time-tracking strategy.", 'Hanjin Group has already had experience in resolving two-year-old conflicts through subsidiary splits.', 'Her brother, Hathawas GS Home Shopping Vice President, was named President-in-Office.', 'The failure of the Real Estate Project Financing (PF) led to a crisis in the non-resident sector.', 'The government has warned of a follow-up response to the rising foreclosure rate in some areas, including Seoul, which has been shut down after the real estate measures.', "So the payroll is 2 million yuan, and it's 1.4 million yuan, and if you give 1.4 million yuan, then 90 percent of that, or 1.26 million yuan, is government subsidized.", 'The activation of cross-border transactions based on the Internet, mobile, etc. is shaking the local customer base.', 'Lotto, which announced the transition to an emergency operating system, announced an aggressive investment plan to in

  3%|▎         | 2/63 [00:19<10:22, 10.20s/it]

['A Financial Directorate official said on 17 May that "if there is a notification of any real estate transactions captured by a government agency, we will separate further fact-checking to see how the funds were actually used or whether there are no problems with the loan regulations".', 'In fact, under the Capital Markets Act, the minimum investment amount of the operating participatory PEF mutual fund invested by the family of the Minister of Finance is 300 million yuan, and the professional investment mutual fund must invest more than 100 million yuan.', 'There are trends in consumption, savings, investments and other economic activities.', 'I thought lime was a fruit name.', 'Seoul City is providing emergency funds to minimize small business casualties due to the novel coronavirus.', 'Bain Capital, Hyal, reportedly has relatively little interest in acquiring.', 'Using machine learning techniques, AI reads the samopond report and assesses the appropriateness of each item and suppor

  5%|▍         | 3/63 [00:27<08:58,  8.98s/it]

...

['Today, it is interpreted that the vice president\'s visit to Samsung was related to the intention of taking over the non-electronic affiliate as the "Samsung General Manager".', 'The majority of the industries are manufacturing, except paper, paper, telecommunications, other manufacturing, nonmetal, printing, media processing, and transportation.', 'The Finance Minister pointed out that out of 51 stock options granted to the stock options trading companies, only eight sales profit realisation companies have increased their annual losses, but the stock options event has increased each year.', 'Real estate experts advise that it is preferable for multi-tenants to register as a tenant before the law is amended.', "The taxpayer's liability for the decline in value is also strengthened if taxpayers deliberately undermine the value of shares, such as large-scale dividends, corporate divisions, and sales transfers.", 'Toss, who fell on the reserve list, had been issuing a free cash withdraw

 98%|█████████▊| 62/63 [09:53<00:09,  9.01s/it]

['The official of the corporate bank said that "the bank president of Yoonyang has ordered various support measures for small and medium-sized enterprises that are suffering from the spread of coronavirus", and "provided effective and active support until the situation is calm".', 'After joining, Jo, who had been doing intensive transactions with retail banks for a long time, naturally raised her credit rating, which was recognized as interest-bearing private property and reduced the interest rate of $30 million to 4.00% from 4.96%.', 'The representative of the office industry, which remained as part of the low-cost ordering processor, said that the minister had crossed the water in the first half of the year with the Chinese factory halt.', 'A Trans-Life official said: "We expect the system of all-party systems to be established and companies to continue to develop in a stable way".', 'At the time of the DLF, some banks had said they would accept the results before the split was held,

100%|██████████| 63/63 [10:02<00:00,  9.56s/it]

['At the founding of the city of Pyongyang, Kim Ji-su, chairman of the congress of the Republic of Korea, and the general association of the executive of the state of Jiangsu, chairman of the board of directors of the congress of the Republic of Korea, and the director of the congress of the state of Korea, were present at the demonstration.', 'The prospect of the most sensitive self-employed loans to the cold war already surpassing 700 trillion dollars suggests that they could be the brainstem of our economy.', 'The role of Ben Bernanke, the chief financial officer during the financial crisis, was decisive.', 'The vice president of the marketing department of the Kimintao Basket Bank said, "We have launched a cross-border insurance platform to increase customer access", and "we will continue to improve the system so that customers can subscribe to the most suitable products for themselves".', 'B said that the market needs more capital and that if it invests, it will give a fixed profi

In [44]:
english_translations_split = [t.split() for t in english_translations]

english_sentences = test_df['영어'].tolist()[:1000]
english_sentences_split = [[s.split()] for s in english_sentences]

print(corpus_bleu(english_sentences_split, english_translations_split))

0.16413727395819905


In [48]:
from transformers.optimization import Adafactor
from transformers import get_constant_schedule_with_warmup
#nllb.cuda();
optimizer = Adafactor(
    [p for p in nllb.parameters() if p.requires_grad],
    scale_parameter=False,
    relative_step=False,
    lr=1e-4,
    clip_threshold=1.0,
    weight_decay=1e-3,
)
scheduler = get_constant_schedule_with_warmup(optimizer, num_warmup_steps=100)

In [54]:
data_train = train_df.copy()
data_train = data_train.filter(['한국어', '영어'], axis=1)


In [60]:
LANGS = [('영어', 'en_Latn'), ('한국어', 'kor_Hang')]

def get_batch_pairs(batch_size, data=data_train[:1000]):
    (l1, long1), (l2, long2) = random.sample(LANGS, 2)
    x, y = [], []
    for _ in range(batch_size):
        item = data.iloc[random.randint(0, len(data)-1)]
        x.append(item[l1])
        y.append(item[l2])
    return x, y, long1, long2

print(get_batch_pairs(1, data_train[:1000]))

(['또 이와 관련, 양국 관계자들이 모여 공동선언문을 발표할 예정이다.'], ['And regarding this, officials from the two countries will also gather to issue a joint declaration.'], 'kor_Hang', 'en_Latn')


In [61]:
batch_size = 16 
max_length = 60 
training_steps = 2000 
losses = [] 
MODEL_SAVE_PATH = '/models'

In [62]:
nllb_new = deepcopy(nllb)
nllb_new.train()
x, y, loss = None, None, None

tq = trange(len(losses), training_steps)
for i in tq:
    x, y, lang1, lang2 = get_batch_pairs(batch_size)
    try:
        tokenizer.src_lang = lang1
        x = tokenizer(x, return_tensors='pt', padding=True, truncation=True, max_length=max_length).to(nllb_new.device)
        tokenizer.src_lang = lang2
        y = tokenizer(y, return_tensors='pt', padding=True, truncation=True, max_length=max_length).to(nllb_new.device)
        y.input_ids[y.input_ids == tokenizer.pad_token_id] = -100

        loss = nllb_new(**x, labels=y.input_ids).loss
        loss.backward()
        losses.append(loss.item())

        optimizer.step()
        optimizer.zero_grad(set_to_none=True)
        scheduler.step()

    except: 
        optimizer.zero_grad(set_to_none=True)
        x, y, loss = None, None, None
        print('error', max(len(s) for s in x + y), e)
        continue

    if i % 1000 == 0:
        print(i, np.mean(losses[-1000:]))

    if i % 1000 == 0 and i > 0:
        nllb_new.save_pretrained(MODEL_SAVE_PATH)
        tokenizer.save_pretrained(MODEL_SAVE_PATH)

  0%|          | 1/2000 [00:15<8:32:07, 15.37s/it]

0 2.1635966300964355


 10%|▉         | 190/2000 [1:13:34<17:24:52, 34.64s/it]